# A minimal version of asprin

The goal of this project is to implement a minimal version of the system *asprin*.

Before you start, we recommend you to read sections 4 and 5 of the paper [1], and 
do the following little project on multi-shot solving:
* [../multi-shot-shell/multi-shot-shell.ipynb](../multi-shot-shell/multi-shot-shell.ipynb)

We introduce *asprin* below, but if you are interested, you can find more information at:
* the paper [2]
* the repository https://github.com/potassco/asprin
* these [slides](https://github.com/potassco/asprin/releases/download/v1/asprin-slides-btw17.pdf)
* the [Potassco Guide](https://github.com/potassco/guide/releases/download/v2.2.0/guide.pdf)


[1] [Kaminski, R., Romero, J., Schaub, T., & Wanko, P. (2023). How to Build Your Own ASP-based System?! Theory Pract. Log. Program., 23(1), 299–361.](https://www.cs.uni-potsdam.de/wv/publications/DBLP_journals/tplp/KaminskiRSW23.pdf)

[2] [Brewka, G., Delgrande, J. P., Romero, J., & Schaub, T. (2023). A general framework for preferences in answer set programming. Artif. Intell., 325, 104023.](https://www.cs.uni-potsdam.de/wv/publications/DBLP_journals/ai/BrewkaDRS23.pdf) 

# The ASP system asprin

In *clingo* we can write logic programs with optimization statements and compute their optimal answer sets.

The system *asprin* extends *clingo* by allowing more general optimization statements: subset preferences, conditional prefereces, their combination, and much more...

A nice feature of *asprin* is that we can define new types of optimization statements simply writing a logic program.

Let's see some examples.

The following *clingo* program:

In [52]:
%%file example1.lp
dom(1..2).
1 { a(X) : dom(X) }.
#show a/1.

Overwriting example1.lp


has 3 answer sets: `{a(1)}`, `{a(2)}` and `{a(1),a(2)}`:

In [66]:
! clingo example1.lp 0 -V0

a(2)
a(1)
a(1) a(2)
SATISFIABLE


If we add a minimize statement:

In [54]:
%%file minimize.lp
#minimize{ 1,X : a(X) }.

Overwriting minimize.lp


then the resulting program has 2 **optimal** answer sets: `{a(1)}` and `{a(2)}`:

In [55]:
! clingo example1.lp minimize.lp --opt-mode=optN 0 -V0 --quiet=1

a(2)
Optimization: 1
a(1)
Optimization: 1
OPTIMUM FOUND


Here, option `--quiet=1` tells clingo to print only the optimal answer sets.

In asprin, we can write this *preference specification*:

In [62]:
%%file preference1.lp
#preference(p1,less(weight)) { 1,X ::     a(X) : dom(X) }.
#preference(p2,      subset) {            a(X) : dom(X) }.

#const o=p1.
#optimize(o).

Overwriting preference1.lp


The **first line** says that the preference statement `p1` is of type `less(weight)` and has the preference element `1,X :: a(X) : dom(X)`.

This statement expresses the same preference as the minimize statement from above, with a slightly different syntax.

An answer set Y is optimal wrt `p1` if there is no answer set X such there are less atoms of `a/1` in X than in Y.

The **second line** says that the preference statement `p2` is of type `subset` and has the preference element `a(X) : dom(X)`.

This expresses a preference for answer sets that are subset-minimal wrt the set of atoms of predicate `a/1`.

An answer set Y is optimal wrt `p2` if there is no answer set X such that the atoms of `a/1` in X are a subset of the atoms of `a/1` in Y.

The **last line** says tells *asprin* to compute optimal answer sets wrt preference statement `o`, and `o` is set to `p1` in the previous line.

##### Formalities

In *asprin*, every preference statement $s$ defines a preference relation $\succeq_s$ between answer sets. 

A preference relation $\succeq_s$ is a transitive and reflexive relation.
Its strict version $\succ_s$ is defined as follows: X $\succ_s$ Y if X $\succeq_s$ Y but Y $\not\succeq_s$ X.

Then, an answer set Y of a program P is optimal wrt $s$ if there is no answer set X of P such that X $\succ_s$ Y.

To use *asprin* in this notebook, we clone it from its repository:

In [ ]:
! git clone https://github.com/potassco/asprin.git

Normally, we would install it with:
* `conda install -c potassco asprin --yes`

but today (23.5.2024) the latest version available via conda is not compatible with the latest version of *clingo* used in this notebook.

Then we can just run *asprin* from the repository:

In [63]:
! python asprin/asprin/asprin.py example1.lp preference1.lp 0

asprin version 3.1.2beta
Reading from example1.lp ...
Solving...
Answer: 1
a(2)
OPTIMUM FOUND
Answer: 2
a(1)
OPTIMUM FOUND

Models       : 2
  Optimum    : yes
  Optimal    : 2
Calls        : 7
Time         : 0.080s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.076s


Observe how we obtain the same optimal answer sets as above with the minimize statement.

We can optimize wrt `p2` changing the constant `o`:

In [64]:
! python asprin/asprin/asprin.py example1.lp preference1.lp -c o=p2 0

asprin version 3.1.2beta
Reading from example1.lp ...
Solving...
Answer: 1
a(2)
OPTIMUM FOUND
Answer: 2
a(1)
OPTIMUM FOUND

Models       : 2
  Optimum    : yes
  Optimal    : 2
Calls        : 7
Time         : 0.072s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.072s


In this case, we also obtain the same optimal answer sets. 

The answer sets are `{a(1)}`, `{a(2)}` and `{a(1),a(2)}`. 

For `{a(1)}` and `{a(2)}`, there is no answer set that is a subset of them wrt `a/1`, since `{}` is not an answer set. 
Hence, both are optimal.

But for `{a(1),a(2)}`, both `{a(1)}` and `{a(2)}` are answer sets that are subsets of it wrt `a/1`.
Hence, `{a(1),a(2)}` is not an optimal answer set.

We can now extend the base program with these rules:

In [70]:
%%file example2.lp
dom(3).
a(3) :- a(2).
a(2) :- a(3).

Overwriting example2.lp


and we obtain 3 answer sets: 

In [71]:
! clingo example1.lp example2.lp 0 -V0

a(1)
a(3) a(2)
a(3) a(2) a(1)
SATISFIABLE


Now only the first one is minimal wrt the minimize statement or `p1`:

In [72]:
! clingo example1.lp example2.lp minimize.lp --opt-mode=optN 0 -V0 --quiet=1

a(1)
Optimization: 1
OPTIMUM FOUND


In [73]:
! python asprin/asprin/asprin.py example1.lp example2.lp preference1.lp 0

asprin version 3.1.2beta
Reading from example1.lp ...
Solving...
Answer: 1
a(1)
OPTIMUM FOUND

Models       : 1
  Optimum    : yes
  Optimal    : 1
Calls        : 4
Time         : 0.071s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.069s


but both `{a(1)}` and `{a(2),a(3)}` are optimal wrt `p2`:

In [74]:
! python asprin/asprin/asprin.py example1.lp example2.lp preference1.lp -c o=p2 0

asprin version 3.1.2beta
Reading from example1.lp ...
Solving...
Answer: 1
a(1)
OPTIMUM FOUND
Answer: 2
a(3) a(2)
OPTIMUM FOUND

Models       : 2
  Optimum    : yes
  Optimal    : 2
Calls        : 7
Time         : 0.073s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.073s


The answer set `{a(2),a(3)}` is optimal because none of its subsets (`{}`, `{a(2)}` or `{a(3)}`) is an answer set.

Observe that `{a(1)}` has less cardinality than `{a(2),a(3)}` but is not a subset of it. 
This lets `{a(2),a(3)}` be optimal wrt `p2` but not wrt `p1`.

This kind of subset preferences are very useful in practice and often show up in real-world applications.

# Task


The task is to implement a minimal version of *asprin*, called *mini-asprin*.

The input to *mini-asprin* should be:
- some *clingo* program including minimize statements, and
- some *preference* program re-interpreting those minimize statements (see below).

Given that input, 
*mini-asprin* should print the answer sets of the *clingo* program 
that are optimal wrt the re-interpretation of the minimize statements. 

For example, if:
* the input consists of the files `example1.lp example2.lp minimize.lp`, and
* the preference program (in `less-weight.lp`) just interprets minimize statements as they are,

then we should obtain the unique optimal answer set `{a(1)}`:

In [24]:
! clingo example1.lp example2.lp minimize.lp --output=reify | python my-mini-asprin.py - less-weight.lp 0

clingo version 5.7.1
Reading from - ...
Solving...
Answer: 1
a(1)
Solving...
OPTIMUM FOUND
Solving...
UNSATISFIABLE

Models       : 1
Calls        : 3
Time         : 0.002s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.002s


But if the preference program (in `subset.lp`) interprets the minimize statement as a subset preference, 
then we should obtain the two optimal answer sets `{a(1)}` and `{a(2), a(3)}`:

In [25]:
! clingo example1.lp example2.lp minimize.lp --output=reify | python my-mini-asprin.py - subset.lp 0

clingo version 5.7.1
Reading from - ...
Solving...
Answer: 1
a(1)
Solving...
OPTIMUM FOUND
Solving...
Answer: 1
a(2) a(3)
Solving...
OPTIMUM FOUND
Solving...
UNSATISFIABLE

Models       : 2
Calls        : 5
Time         : 0.004s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.004s
